In [87]:
import requests
import bs4
import json
import re
import collections

In [88]:
# # https://hayday.fandom.com/wiki/Goods_List

# url = "https://hayday.fandom.com/wiki/Goods_List"
# response = requests.get(url)
# response.raise_for_status()
# soup = bs4.BeautifulSoup(response.text, "html.parser")

In [89]:
# # get table
# table = soup.find("table", {"class": "wikitable"})
# rows = table.find_all("tr")
# # first row is header
# header = rows[0].find_all("th")
# header = [h.text.strip().lower() for h in header]

# data = []
# for row in rows[1:]:
#     cols = row.find_all("td")
#     # replace npsp with space
#     cols = [re.sub(r"\xa0", " ", c.text).strip().lower() for c in cols]
#     data.append(dict(zip(header, cols)))
# # save to json
# with open("hayday_goods.json", "w") as f:
#     json.dump(data, f, indent=2)

In [90]:
# data = json.load(open("hayday_goods.json"))
# # parse needs for each good
# for item in data:
#     pattern = r'(\b[\w\s]+\b)\s*\((\d+)\)'
#     needs = item["needs"]
#     matches = re.findall(pattern, needs)
#     if matches:
#         item["needs"] = [{'name': match[0], 'quantity': int(match[1])} for match in matches]
#     else:
#         item["needs"] = {}

# # parse source
# for item in data:
#     source = item["source"]
#     # remove anything in parentheses
#     source = re.sub(r"\(.*?\)", "", source)
#     source = source.strip()
#     item["source"] = source

# # parse time
# for item in data:
#     time = item["time"]
#     #  remove anything include and after \u2605\u2605\u2605
#     time = re.sub(r"\u2605\u2605\u2605.*", "", time)
#     time = time.strip()
#     regex = re.compile(r"(\d+) (d|h|min)")
#     total_time = 0
#     for match in regex.finditer(time):
#         quantity, unit = match.groups()
#         quantity = int(quantity)
#         if unit == "d":
#             total_time += quantity * 24 * 60 * 60
#         elif unit == "h":
#             total_time += quantity * 60 * 60
#         elif unit == "min":
#             total_time += quantity * 60
#     item["time"] = total_time
# # save to json
# with open("hayday_goods.json", "w") as f:
#     json.dump(data, f, indent=2)

In [91]:
data = json.load(open("hayday_goods.json"))
source = set()
for item in data:
    source.add(item["source"])
source = collections.Counter(source)
source['field']=10
source['cow']=4
source_id = {}
id = 0
for key in source:
    source_id[key] = []
    for _ in range(source[key]):
        id += 1
        source_id[key].append(id)

def find_dependencies(name):
    result = None
    for item in data:
        if item["name"] == name:
            result = item.copy()
            break
    if result is None:
        print(f"{name} not found")
        return
    needs = []
    for need in result["needs"]:
        if need["name"] == name:
            continue
        dependency = find_dependencies(need["name"])
        if dependency is not None:
            needs.append(dependency)
    result["needs"] = needs
    return result

def make_task(good_names):
    id = 0
    tasks = []
    def make_task_helper(root,job_id):
        nonlocal id
        id += 1
        task = {"id": id, "name": root["name"], "duration": root["time"], "dependent": [],'machine':source_id[root["source"]],'job_id':job_id}
        tasks.append(task)
        for need in root["needs"]:
            task["dependent"].append(id + 1)
            make_task_helper(need,job_id)

    for job_id in range(len(good_names)):
        for name in good_names[job_id]:
            root = find_dependencies(name)
            make_task_helper(root,job_id)
    return tasks

tasks = make_task([["cheesecake",'baked potato','strawberry cake','casserole'],
                   ['spicy pizza','frutti di mare pizza','beetroot salad','cheese omelet'],
                   ['wheat','wheat','wheat','wheat','wheat','wheat',]+['corn']*100])

# save source_id (machine) and tasks to json
with open("hayday_tasks.json", "w") as f:
    json.dump({"machine": source_id, "tasks": tasks}, f)

In [92]:
# load 
tasks

[{'id': 1,
  'name': 'cheesecake',
  'duration': 14400,
  'dependent': [2, 7],
  'machine': [49],
  'job_id': 0},
 {'id': 2,
  'name': 'cheese',
  'duration': 3600,
  'dependent': [3],
  'machine': [53],
  'job_id': 0},
 {'id': 3,
  'name': 'milk',
  'duration': 3600,
  'dependent': [4],
  'machine': [1, 2, 3, 4],
  'job_id': 0},
 {'id': 4,
  'name': 'cow feed',
  'duration': 600,
  'dependent': [5, 6],
  'machine': [70],
  'job_id': 0},
 {'id': 5,
  'name': 'corn',
  'duration': 300,
  'dependent': [],
  'machine': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
  'job_id': 0},
 {'id': 6,
  'name': 'soybean',
  'duration': 1200,
  'dependent': [],
  'machine': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
  'job_id': 0},
 {'id': 7,
  'name': 'cookie',
  'duration': 3600,
  'dependent': [8, 10, 14],
  'machine': [71],
  'job_id': 0},
 {'id': 8,
  'name': 'brown sugar',
  'duration': 1200,
  'dependent': [9],
  'machine': [45],
  'job_id': 0},
 {'id': 9,
  'name': 'sugarcane',
  'duration': 1800,
  'de